# Intro to AnnoMate Reviewers

All `Reviewer` types in this package follow the same general framework for setting up the dashboard. 

This notebook will walk you through how to interact with a basic reviewer, which extends to other `Reviewer` types.

If you want to know how to **create** your own custom reviewer, see `Developer_AnnoMate_Tutorial`

## Installation

Using pypi (recommended)
1. Create an environment: `conda create --name <my-env> --file requirements.txt python==3.8`
1. Activate environment: `conda activate <my-env>`
1. Install AnnoMate: `pip install AnnoMate`

Using git:
1. Download the repository: `git clone git@github.com:getzlab/AnnoMate.git` 
1. `cd AnnoMate`
1. Create an environment: `conda create --name <my-env> --file requirements.txt python==3.8`
1. Activate environment: `conda activate <my-env>`
1. Install package: `pip install -e .`

Running from docker container: You need to open at least 2 ports if you are using Mac or Windows:
1. A port to open jupyter lab (in this case, `<jupyter_port>`)
2. A port to open the dash app (`<dash_port>`)
```
docker run -it -p <jupyter_port>:<jupyter_port> -p <dash_port>:<dash_port> gcr.io/broad-getzlab-workflows/annomate:dev_branch_v385
cd AnnoMate/tutorial_notebooks
jupyter lab --ip 0.0.0.0 --port <jupyter_port> --no-browser --allow-root
```

## Running this notebook

Make sure to update your kernel to the conda environment `<my-env>` you just created.


## Imports
For this tutorial, we just need a few packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os

## The high level steps

There are 7 steps to get started reviewing in your jupyter notebook:
1. Pick your reviewer
1. Instatiate the selected reviewer
1. Set up the review data
1. Set up the app
1. Set up defaults settings
1. Run your reviewer!

It may seem to be a lot of steps, but it will actually only look something like this

First, we will go through some basic terminology. Next, we will walk through each step below using basic functionality. In the last section, we will go through more advanced options for each step.

## General Terminology

- A **Subject Type** refers to what "level" or "item" you are reviewing. For example, you may be reviewing samples (ie purity), participants (ie clinical data or comprehensive data), mutations (checking if it is an artifact), etc.

- A **Subject** is an individual item you are currently reviewing or manually annotating.

- A **Reviewer** is the class in the `AnnoMate` package that manages the data you want to review and implements a user interface for you to review each **Subject** one at a time, view/interact its corresponding data, and make annotations.

- An **Annotation** is some value associated with a given **Review Subject** given some kind of analysis or manual observation. A Review Subject can have multiple annotations. An Annotation may have certain parameters about what kinds of values are allowed (a list of options, or a range of values, etc.)

- **Data** in this context is actually an object that stores collection of tables. It includes the information to review for your Review Subject type (ie sample table from Terra), and the annotations you will eventally make for it. Its purpose is to "freeze" the data you are reviewing and couple the data with the annotations you make. Once this object is made, it will store all the data in a pickle file. From then on, only your annotations can be (easily) modified in a specific way (by the `ReviewData` class)

- An **ReviewDataApp** is a user interface to display your Data, such as displaying charts or graphs of the information associated with your Review Subject Type, and a way to add annotations for the currently displayed Review Subject. In this package, we use `plotly.dash` to create dashboards for this purpose.

![](https://github.com/getzlab/AnnoMate/blob/master/images/Reviewer%20Diagram.jpg)


## Basic Reviewer Run

### 1. Select your reviewer

Check out the ReviewerCatalog at [example_notebooks/ReviewerCatalog.ipynb](https://github.com/getzlab/AnnoMate/blob/master/example_notebooks/ReviewerCatalog.ipynb) for different reviewers.

For this tutorial, we will use `MyCustomReviewer`. Go to `AnnoMate/Reviewers/` to view other available prebuilt options. 

**This tutorial only applies to any `Reviewer` that inherits from `AnnoMate.ReviewerTemplate`**

Let's import `MyCustomReviewer`. This reviewer is built to review some dummy sample data. 

In [3]:
from AnnoMate.Reviewers.ExampleReviewer import ExampleReviewer

### 2. Instantiate the selected reviewer

This step is super simple, just create an object, no parameters required

In [4]:
my_reviewer = ExampleReviewer()

### 3. Set up the review data

At this step you give `my_reviewer` data you want to start reviewing. 

The type of data you need depends on the type of reviewer you are using. You can see what data it requires by typing the reviewer's `.set_review_data()` in a cell, place your cursor at the end and press `Shift+Tab`.

You will see these following required parameters:
- `data_path: pathlib.Path`: Path to directory to save your data.
- `description: str`: describe what data you are reviewing. It's a good idea to also include why

The remaining parameters are optional, and allows you to prefill the annotation and history information. See X section below for more information.

Depending on the reviewer you may additional arguments or tables (`**kwargs`), which should be included in the docstring. Alternatively, you can look at the source code directly.

`MyCustomReviewer` only requires a single dataframe.

> What is the `data_path` for? All the data you review is used to create a `Data` object. This object saves all the data you want to review, and a corresponding `annot_df` and `history_df` dataframes. This data is all stored in this directory in a file named `data.pkl`. There will also be an automatically generated `metadata_config.yaml` file in this directory that stores information about the current state of the reviewer. **The user should not touch this file**.
> - The `annot_df` dataframe stores the annotations you are recording for each item you are reviewing
> - The `history_df` dataframe stores all the changes that have been made to the annotation table
>
> The `description` parameter is for you to describe the source of the data and what the review process is for.
>
> **The purpose of this is to always couple the annotations with the data that was actually used to review it.**


In [5]:
# Load data
fn = 'example_data/AnnoMate_Tutorial/data_to_review_example.tsv'
df = pd.read_csv(fn, sep='\t')
df = df.set_index('sample_id')
df.head()

,gender,age,tissue_origin,treatments_file,mutations_file
sample_id,,,,,
sample_0,female,57,skin,./example_data/AnnoMate_Tutorial/treatments/sa...,./example_data/AnnoMate_Tutorial/mut_vafs/samp...
sample_1,male,66,breast,./example_data/AnnoMate_Tutorial/treatments/sa...,./example_data/AnnoMate_Tutorial/mut_vafs/samp...
sample_2,male,65,lung,./example_data/AnnoMate_Tutorial/treatments/sa...,./example_data/AnnoMate_Tutorial/mut_vafs/samp...
sample_3,male,66,skin,./example_data/AnnoMate_Tutorial/treatments/sa...,./example_data/AnnoMate_Tutorial/mut_vafs/samp...
sample_4,male,48,bone marrow,./example_data/AnnoMate_Tutorial/treatments/sa...,./example_data/AnnoMate_Tutorial/mut_vafs/samp...


In [6]:
mut_df = pd.read_csv(df.iloc[0]['mutations_file'], sep='\t')
mut_df.head()

,gene,vaf,sample_id,cov,t_alt_count,t_ref_count
0,gene_0,0.334632,sample_0,110,36,74
1,gene_1,0.246997,sample_0,106,26,80
2,gene_2,0.357634,sample_0,133,47,86
3,gene_3,0.789236,sample_0,85,67,18
4,gene_4,0.654362,sample_0,146,95,51


In [7]:
output_pkl_path = './example_reviewer_data'
my_reviewer.set_review_data(data_path=output_pkl_path, 
                            description='Intro to reviewers review session part 2',
                            sample_df=df,
                            preprocessing_str='Testing preprocessing')

/Users/cchu/opt/anaconda3/envs/test/lib/python3.8/site-packages/AnnoMate/ReviewDataInterface.py:41: UserWarning: Loading existing data pkl file
  warnings.warn(f"Loading existing data pkl file")


This cell will create your data path directory if it doesn't exist already, creating the before mentioned pickle and config files with it. When these files are initially generated the reviewer is in **test mode**. This means when the above cell is rerun:
- Changes to input data (in this case sample_df) will be update in the data object
- Changes to annototations / annotation history made in the dashboard will not be saved and be reset 

Once test mode* is disabled, rerunning the above cell should give you a warning. Now, any time that directory `./example_reviewer_data` is passed to *any* reviewer's `set_review_data()` parameter `data_path`, it will simply load whatever is currently in the `data.pkl` file. It will NOT update any of its attributes to whatever the value of the other parameters (in this case, `description`, `sample_df`, `preprocessing_str`, etc.).

> **Why do this?** Often times we use `dalmatian` to pull data from Terra workspaces. Sometimes the data in Terra changes because we run workflows multiple times with different parameters. We want to avoid losing what data we were originally looking at to produce the annotations we currently have.
>
> If you do want to "update" your data, make a new session pointing to a different pickle file path. More on this later.


You can also see that changes were made to the input dataframe

*See section 6 to learn how to disable test mode

In [8]:
my_reviewer.list_data_attributes()

dict_keys(['index', 'description', 'annot_col_config_dict', 'annot_df', 'history_df', 'df'])

In [9]:
my_reviewer.get_data_attribute('df').head()

,gender,age,tissue_origin,treatments_file,mutations_file,new_column
sample_id,,,,,,
sample_0,female,57,skin,./example_data/AnnoMate_Tutorial/treatments/sa...,./example_data/AnnoMate_Tutorial/mut_vafs/samp...,Testing preprocessing
sample_1,male,66,breast,./example_data/AnnoMate_Tutorial/treatments/sa...,./example_data/AnnoMate_Tutorial/mut_vafs/samp...,Testing preprocessing
sample_2,male,65,lung,./example_data/AnnoMate_Tutorial/treatments/sa...,./example_data/AnnoMate_Tutorial/mut_vafs/samp...,Testing preprocessing
sample_3,male,66,skin,./example_data/AnnoMate_Tutorial/treatments/sa...,./example_data/AnnoMate_Tutorial/mut_vafs/samp...,Testing preprocessing
sample_4,male,48,bone marrow,./example_data/AnnoMate_Tutorial/treatments/sa...,./example_data/AnnoMate_Tutorial/mut_vafs/samp...,Testing preprocessing


### 4. Set up the app

Depending on the reviewer, you may have options to customize how the dashboard app will be displayed. In this example, we need to provide information about which columns in the input table we want to use as the mutation file and which of the columns in the sample table to display.

Feel free to change the `sample_cols` parameter to any set of columns or order from the input dataframe above.

In [10]:
my_reviewer.set_review_app(mut_file_col='mutations_file', 
                           sample_cols=['gender', 'age', 'tissue_origin'])

### 5. Set up default settings

For now, lets just set up the default annotations to record and other settings. More details in the Advanced section below.

In [11]:
my_reviewer.set_default_review_data_annotations_configuration()
my_reviewer.set_default_autofill()

In [12]:
my_reviewer.get_annot().head()

,Notes,Flag
sample_0,,Remove
sample_1,,Remove
sample_2,Nothing to say,Keep
sample_3,,Remove
sample_4,Not good,Remove


In [13]:
my_reviewer.get_history().head()

,index,timestamp,source_data_fn,Notes,Flag
0,sample_0,2024-05-29 11:21:53.051037,./example_reviewer_data/data.pkl,,Remove
0,sample_1,2024-05-29 11:21:56.466574,./example_reviewer_data/data.pkl,,Remove
0,sample_2,2024-05-29 11:22:02.213915,./example_reviewer_data/data.pkl,Nothing to say,Keep
0,sample_3,2024-05-29 11:22:05.466076,./example_reviewer_data/data.pkl,,Remove
0,sample_4,2024-05-29 11:22:12.998607,./example_reviewer_data/data.pkl,Not good,Remove


### 6. Run the reviewer!

You can run the app inside the notebook (`mode='inline'`, the default) or in a separate window (`mode='external'` or `mode='tab'`).

If you are running your notebook in a VM, you will need to create an ssh connection, and specify the host and port address

In [14]:
my_reviewer.run(collapsable=True, port=8055, mode='tab')

Setting auto_export_path to ./example_reviewer_data/data.auto_export
Using ./example_reviewer_data/data.auto_export for auto exporting.
Dash app running on http://0.0.0.0:8055/


<IPython.core.display.Javascript object>

> **If you are using a VM:** Make sure to open a ssh tunnel to the specified port number
> - `~C` to open the ssh prompt (will look like `ssh>`)
> - Type and enter `-L <port number>:localhost:<port number>`

> **If you are running from a docker container:** Make sure to open the port for accessing the dash app (`reviewer.run(port=<new_port_number>, ...)`) when you open the container
> - `docker run -it -p <jupyter_port>:<jupyter_port> -p <dash_port>:<dash_port> gcr.io/broad-getzlab-workflows/annomate:dev_branch_v385`
> - You can add more ports by adding more `-p <port_number>:<port_number>` arguments
> - If you are on Linux, you can replace `-p` arguments with `--network host`

You can set `collapsable=False` if you do not want collapsable components.

Go ahead and "review" some of the data by selecting different rows in the drop down menu and inputting annotations into the input form at the top left. Press `submit`, and you will see your inputs update the history table. If you change your annotations for that sample again, the annotation table will keep the most recent change, but you will see in the history table it will show both your new and previous annotations.


For this exercise, fill in annotations for at least 8 samples, where at least four of them you annotate `Remove` for the `Flag` annotation

You can view your progress by accessing the annotation table or history table:

In [15]:
# only viewing samples with annotations
my_reviewer.get_annot().head()

,Notes,Flag
sample_0,,Remove
sample_1,,Remove
sample_2,Nothing to say,Keep
sample_3,,Remove
sample_4,Not good,Remove


In [16]:
my_reviewer.get_history()

,index,timestamp,source_data_fn,Notes,Flag
0,sample_0,2024-05-29 11:21:53.051037,./example_reviewer_data/data.pkl,,Remove
0,sample_1,2024-05-29 11:21:56.466574,./example_reviewer_data/data.pkl,,Remove
0,sample_2,2024-05-29 11:22:02.213915,./example_reviewer_data/data.pkl,Nothing to say,Keep
0,sample_3,2024-05-29 11:22:05.466076,./example_reviewer_data/data.pkl,,Remove
0,sample_4,2024-05-29 11:22:12.998607,./example_reviewer_data/data.pkl,Not good,Remove


### Disable test mode

On initial load, you'll notice a `Freeze data` button. Once you are happy with your input data and are ready to start reviewing your data, press this button to disable test mode and freeze your data. Note that once this button is pressed, you will not be able to unfreeze your data / revert back to test mode.

Now you can export this table to a file that you can then share, upload to Terra, or use for further analysis.

## Export your data

This is highly recommended to run periodically. Exporting will write the `annot_df` and `history_df` dataframes to a tsv file in the directory specified. 

We include a boolean parameter to `export_by_day`, which will create a sub-directory inside the input directory labeled by the current date (YYYY-MM-DD) and save the current state of your review in that folder.

We also include a boolean parameter `dry_run` so you can see where the data will be exported.

In [17]:
export_dir = f'data/example_reviewer_data'
if not os.path.exists(export_dir):
    os.mkdir(export_dir)
    
my_reviewer.export_data(export_dir, export_by_day=True)

Making new directory data/example_reviewer_data/2024-06-03
Export directory will be data/example_reviewer_data/2024-06-03. Nothing exported yet.


In [18]:
# Actually saving the data
my_reviewer.export_data(export_dir, export_by_day=True, dry_run=False)

index is not a dataframe. Not exporting.
description is not a dataframe. Not exporting.
annot_col_config_dict is not a dataframe. Not exporting.
Saving annot_df to data/example_reviewer_data/2024-06-03/annot_df.tsv
Saving history_df to data/example_reviewer_data/2024-06-03/history_df.tsv
Saving df to data/example_reviewer_data/2024-06-03/df.tsv
Exported to data/example_reviewer_data/2024-06-03


/Users/cchu/opt/anaconda3/envs/test/lib/python3.8/site-packages/AnnoMate/ReviewerTemplate.py:513: UserWarning:

Directory data/example_reviewer_data/2024-06-03 already exists



## Advanced Reviewer Run

We will go through each step again, but show how you can further customize your reviewer for your needs.

Let's suppose you are done reviewing all the samples in `my_reviewer` above, and now I want to do some more exploration on the samples I decided to `Remove`.

### 1. and 2. Pick and instantiate your reviewer

We will just use the same one as before, but this time create a separate reviewer

In [19]:
my_reviewer_2 = ExampleReviewer()

### 3. Set up the review data

For this exercise, let's identify which samples you "removed" from the previous review. 


In [20]:
keep_samples_index = \
    my_reviewer.get_annot().loc[
        my_reviewer.get_annot()['Flag'] == 'Remove'
    ].index.tolist()

print(keep_samples_index)

['sample_0', 'sample_1', 'sample_3', 'sample_4']


It would be very useful in this new review session to know why I initially thought I should remove those samples. We can include the previous annotation and history data in this new review session. 

You may have seen in `ReviewerTemplate.set_review_data` there were several other parameters that were not discussed. Those parameters are to allow you to "pre-fill" annotations and history.

There are two main ways to do this:

1. **Manually input the `annot_df`, `annot_col_config_dict`, and `history_df` yourself.** This may be appropriate if you have some post processing done on the annotations done separately, or you only have access to an `annot_df` but no history.
1. **Preferred: Load existing data pickle object or exported data files** If you are just continuing review from a previous review session. You can do this by either passing in the path to the data pickle file (`load_existing_data_pkl_fn`), or to a directory that contains the exported tables from a review session (`load_existing_exported_data_dir`). 

> The latter method is preferred because the history tables include a column that indicates the source of the 
> annotations (column `source_data_fn`), which allows you to easily go back to that review session and read the 
> description. A work around is to manually generate your own history table with the columns corresponding to your
> input annotation table, plus `['index', 'timestamp', 'source_data_fn']`. 


**NOTE**: you still need to set up the input data you want to review yourself. Often times new or updated data is available. If you want to use the data you used for original review sessions, you can pull directly from the data object or the exported files. 


For this exercise, let's pass in the pickle file directly.

In [21]:
new_output_path = './example_reviewer_data_removed'
my_reviewer_2.set_review_data(
    index=keep_samples_index,
    data_path=new_output_path,
    description="Reviewing more data to see if I can explain why these samples should be removed.",
    sample_df=my_reviewer.get_data_attribute('df'), # reuse exactly the same data as before
    load_existing_data_pkl_fn=f'{output_pkl_path}/data.pkl',
    preprocessing_str='New data'
)


Loading data from previous review with pickle file


In [22]:
my_reviewer_2.get_annot()

/Users/cchu/opt/anaconda3/envs/test/lib/python3.8/site-packages/AnnoMate/ReviewerTemplate.py:477: UserWarning:

Data is not frozen. Annotations will not be saved. Please freeze data in the dashboard to save annotations.



,Notes,Flag
sample_0,,Remove
sample_1,,Remove
sample_3,,Remove
sample_4,Not good,Remove


### 4. Set up the app

In the previous review session, all we saw was a mutation table. What if we also want to view the treatment data?

You can add additional tables and graphs to the dashboard.

First, run `set_review_app` like before. Note that if you have updated files to plot, make sure you reference the correct column name. In this case, we are still reviewig the original files in `mutations_file`.

In [23]:
my_reviewer_2.set_review_app(
    mut_file_col='mutations_file', 
    sample_cols=['gender', 'age', 'tissue_origin', 'mutations_file']
)


Then you can add your own `AppComponent`. For more advanced features, see `Developer_AnnoMate_Tutorial.ipynb`.

For now, let's just use the built in function to add a table.

In [24]:
pd.read_csv(my_reviewer_2.get_data_attribute('df').iloc[0]['treatments_file'], sep='\t')

,treatment_name,response,tx_start
0,anti-TNF,remission,95
1,anti-TNF,remission,183
2,Anthracycline,remission,203


In [25]:
my_reviewer_2.app.add_table_from_path(
    data_table_source='df', # reference which table in the Data object to use.
    table_title='Treatment file',
    component_id='treatment-component-id',
    table_fn_col='treatments_file', 
    table_cols=['treatment_name', 'response', 'tx_start']
)


## 5. Set up settings

Above, we just used default settings already implemented for us. This included what annotations to record and how they are displayed in the app. 

To create your own annotations, use `add_review_data_annotation(annot_name: str, data_annot: DataAnnotation)` for each annotation you want to use. This will create new columns in the `review_data.data.annot_df` dataframe if `annot_name` does not already exist. If it does, it will update the annotation's metadata with `data_annot`, which specifies information about the data type of the annotation (string, float, etc.), valid options, and default values. 

Similarly, you can specify or change how your annotation inputs will be displayed in the app with `add_annotation_display_component(name: str, annot_display_component: AnnotationDisplayComponent)`. 
- `name` must refer to an annotation column in the `annot_df` dataframe (determined with `add_review_data_annotation()` or default settings). 
- `annot_display_component` is an `AnnotationDisplayComponent` object. Examples include text area (`TextAnnotationDisplay()`), checklist (`ChecklistAnnotationDisplay()`), etc. See more options in `AnnoMate.AnnotationDisplayComponent`. For each`AnnotationDisplayComponent` object, you can set a default display value to prefill the annotation panel with a valid option using the `default_display_value` parameter.


Let's keep the default configuration, but make some modifications:
1. Add an additional option "Report" for the existing "Flag" annotation and set a default value
1. Modify the display of `Notes` to just be a single text line
1. Add an annotation to record the current color of the histogram

**NOTE**: Once those annotations are added, they will stay. If you already made annotations and you change the data type or valid values afterwards, the reviewer will check that existing values are valid. If not, you will not be able to make those changes to the configuration.

In [26]:
from AnnoMate.Data import DataAnnotation
import AnnoMate.AnnotationDisplayComponent as adc
import numpy as np

In [27]:
my_reviewer_2.set_default_review_data_annotations_configuration()

In [28]:
# 1. Update all the options for the 'Flag' annotation
my_reviewer_2.add_review_data_annotation(
    annot_name='Flag', 
    review_data_annotation=DataAnnotation(
        annot_value_type='string', 
        options=['Keep', 'Remove', 'Report'], 
    )
)
my_reviewer_2.add_annotation_display_component(
    annot_name='Flag', 
    annot_display_component=adc.RadioitemAnnotationDisplay(default_display_value='Keep')
)

# 2. Update the display for the 'Notes' annotation
my_reviewer_2.add_annotation_display_component(
    annot_name='Notes', 
    annot_display_component=adc.TextAnnotationDisplay()
)

# 3. Add a histogram color annotation
my_reviewer_2.add_review_data_annotation(
    annot_name='Histogram color', 
    review_data_annotation=DataAnnotation(
        annot_value_type='string', 
        options=['red', 'green', 'blue'], 
    )
)
my_reviewer_2.add_annotation_display_component(
    annot_name='Histogram color', 
    annot_display_component=adc.SelectAnnotationDisplay()
)


Then, let's add more annotations to explore some of other predefined `DataAnnotation` and `AnnotationDisplayComponent` options

In [29]:
# Text area
my_reviewer_2.add_review_data_annotation(
    annot_name='A Number', 
    review_data_annotation=DataAnnotation(
        annot_value_type='float', 
    )
)
my_reviewer_2.add_annotation_display_component(
    annot_name='A Number', 
    annot_display_component=adc.NumberAnnotationDisplay())

# Radioitem: 
my_reviewer_2.add_review_data_annotation(
    annot_name='A radioitem', 
    review_data_annotation=DataAnnotation(
        annot_value_type='string', 
        options=['Option a', 'Option b', 'Option c']
    )
)
my_reviewer_2.add_annotation_display_component(
    annot_name='A radioitem', 
    annot_display_component=adc.RadioitemAnnotationDisplay())

# Checklist: only compatible with multi type (save a list of options)
my_reviewer_2.add_review_data_annotation(
    annot_name='A checklist', 
    review_data_annotation=DataAnnotation(
        annot_value_type='multi', 
        options=['Option 1', 'Option 2', 'Option 3']
    )
)
my_reviewer_2.add_annotation_display_component(
    annot_name='A checklist', 
    annot_display_component=adc.ChecklistAnnotationDisplay(default_display_value=['Option 1']))


# Dropdown/Select
my_reviewer_2.add_review_data_annotation(
    annot_name='A dropdown menu', 
    review_data_annotation=DataAnnotation(
        annot_value_type='string', 
        options=['Option A', 'Option B', 'Option C']
    )
)
my_reviewer_2.add_annotation_display_component(
    annot_name='A dropdown menu', 
    annot_display_component=adc.SelectAnnotationDisplay())

# Multiselect Dropdown menu
my_reviewer_2.add_review_data_annotation(
    annot_name='A multi-select dropdown menu', 
    review_data_annotation=DataAnnotation(
        annot_value_type='multi', 
        options=['Option x', 'Option y', 'Option z']
    )
)
my_reviewer_2.add_annotation_display_component(
    annot_name='A multi-select dropdown menu', 
    annot_display_component=adc.MultiValueSelectAnnotationDisplay())


Sometimes you may not want to review all the annotations in the annotation table in the dashboard. There are two ways to do this:
- Do not run `reviewer.set_default_review_data_annotations_configuration()`, which will add all the default annotations and displays. However, you would then need to add annotation columns yourself if you did not load directly from another review session
- You can remove any pre-built annotation display components by referencing the corresponding annotation name. **NOTE** that this does NOT delete anything in the annotation table. It will only change the annotation panel input options in the dashboard.

In [30]:
my_reviewer_2.add_review_data_annotation(
    annot_name='Test removing annotation', 
    review_data_annotation=DataAnnotation(
        annot_value_type='float', 
    )
)
my_reviewer_2.add_annotation_display_component(
    annot_name='Test removing annotation', 
    annot_display_component=adc.NumberAnnotationDisplay())

'Test removing annotation' in my_reviewer_2.annot_app_display_types_dict.keys()

True

In [31]:
removed_annot_display_component = my_reviewer_2.remove_annotation_display_component('Test removing annotation')
'Test removing annotation' in my_reviewer_2.annot_app_display_types_dict.keys()

False

Annotation "Test removing annotation" will still be in the annotation dataframe. However, since there is no associated display component it's value will not be displayed.

### 5.1 Autofill

Sometimes there are parts of the dashboard that can calculate annotations on the fly. In this case, say we want to record what color the histogram plot was last plotted with. It would be tedious and error prone for a reviewer to have to keep copying data into the correpsonding input. 

Autofill allows the you to simply press a button and it will take the data in the dashboard from specified components and fill the annotation input panel for you.

Depending on the app, there will already be a default autofill setting. For those, just call 
```
my_reviewer_2.set_default_autofill()
```

In this case, `ExampleReviewer` has no autofill specified. You can add your own using `add_autofill()`. This is also useful when you have added your own custom components.

You may have to look at the source code to see what components were added and what layout components have values you can access.


In [32]:
from dash.dependencies import State
my_reviewer_2.add_autofill(
    autofill_button_name='Mut vafs',
    fill_value=State('mut-figure-color-radioitem', 'value'),
    annot_name='Histogram color'
)

## 6. Run the app!

In [33]:
my_reviewer_2.run(port=8052, mode='tab')

Setting auto_export_path to ./example_reviewer_data_removed/data.auto_export
Using ./example_reviewer_data_removed/data.auto_export for auto exporting.
Dash app running on http://0.0.0.0:8052/


/Users/cchu/opt/anaconda3/envs/test/lib/python3.8/site-packages/AnnoMate/ReviewDataApp.py:738: UserWarning:

You are in test mode. Your data will not be saved.



<IPython.core.display.Javascript object>

### Navigate data to review inside the dashboard

Pass in a dataframe with the index that matches the index from the Data object to also traverse your data through a table. You specify which columns you want to include, in addition to the columns in your annotation table which are added automatically.

In [34]:
my_reviewer_2.run(
    review_data_table_df=df.loc[keep_samples_index, ['gender', 'age', 'tissue_origin']], 
    review_data_table_page_size=7,
    port=8052,
    mode='tab'
)

Setting auto_export_path to ./example_reviewer_data_removed/data.auto_export
Using ./example_reviewer_data_removed/data.auto_export for auto exporting.
Dash app running on http://0.0.0.0:8052/


/Users/cchu/opt/anaconda3/envs/test/lib/python3.8/site-packages/AnnoMate/ReviewDataApp.py:738: UserWarning:

You are in test mode. Your data will not be saved.



<IPython.core.display.Javascript object>

In [35]:
my_reviewer_2.get_annot()

/Users/cchu/opt/anaconda3/envs/test/lib/python3.8/site-packages/AnnoMate/ReviewerTemplate.py:477: UserWarning:

Data is not frozen. Annotations will not be saved. Please freeze data in the dashboard to save annotations.



,Notes,Flag,Histogram color,A Number,A radioitem,A checklist,A dropdown menu,A multi-select dropdown menu,Test removing annotation
sample_0,,Remove,,NaN,,,,,NaN
sample_1,,Remove,,NaN,,,,,NaN
sample_3,,Remove,,NaN,,,,,NaN
sample_4,Not good,Remove,,NaN,,,,,NaN


You should see that the new inputs was added to the annotation panel. Additionally, the new table with treatment data is added to the bottom.

Select a sample from the dropdown and change the color of the histogram. 

Then, go back up to the annotaiton panel and press the `Mut val` button. This should autofill the `Histogram color` input to the same color you selected.

### Auto export or "backup" your annotations

The data is automatically saved to a pickle object. However, you may want to maintain a back up copy of your data that is independet of the pickle file. Sometimes if you make environment changes, pickle may not be able to load the original data object and your annotations may be hard to retrieve.

It's recommended to set `auto_export=True` and set a directory (local or bucket url) to save at minimum your annotation and history tables, which is set by default. If you have additional attributes from the data object you want to save (`reviewer.get_data_attributes()`), you can explicitly pass a list to `attributes_to_export`.

If you do not set `auto_export_path`, it will automatically create a directory inside the path passed to `data_path` in `reviewer.set_review_data(...)`. 

In [36]:
my_reviewer_2.run(
    port=8052,
    auto_export=True,
)

Setting auto_export_path to ./example_reviewer_data_removed/data.auto_export
Using ./example_reviewer_data_removed/data.auto_export for auto exporting.
Dash app running on http://0.0.0.0:8052/


/Users/cchu/opt/anaconda3/envs/test/lib/python3.8/site-packages/AnnoMate/ReviewDataApp.py:738: UserWarning:

You are in test mode. Your data will not be saved.



### Change order of components

You may want to exclude rendering certain components from a pre-built reviewer. Or, you may have added a new component and you want to put it adjacent to a particular existing component. By default, the order of the components are displayed in order in which they are added (via `app.add_component(...)`). However, you can change the order and/or exclude certain components to render when running the reviewer.

View all the component names:

In [37]:
my_reviewer_2.app.more_components.keys()

odict_keys(['sample-info-component', 'Maf file', 'Mut vafs', 'Treatment file'])

Specify a new order and/or subset of components to display

In [38]:
my_reviewer_2.run(
    port=8052,
    components_name_order=['sample-info-component', 'Treatment file', 'Mut vafs']
)

Setting auto_export_path to ./example_reviewer_data_removed/data.auto_export
Using ./example_reviewer_data_removed/data.auto_export for auto exporting.


/Users/cchu/opt/anaconda3/envs/test/lib/python3.8/site-packages/AnnoMate/ReviewDataApp.py:738: UserWarning:

You are in test mode. Your data will not be saved.



Dash app running on http://0.0.0.0:8052/


**NOTE**: Some reviewers may have components that listen to each other, so if you keep one but not the other, some of the predefined interactivity may lead to error messages. You may run into similar errors for autofill buttons which may be pulling data from components that are excluded. 

Also, the data needed to rendered excluded components will still be required when setting the review data (`reviewer.set_review_data(...)`).

# Final words

You can always go back and change the parameters of setting the app and annotation configurations, adding or editing the components of the app, etc.

The one thing that will NOT change after disabling test mode is `set_review_data()`. This review data can only be frozen once per `data_path`. You may re-set your review data as much as you'd like in test mode, but once you click the 'Freeze data' button, disabling test mode, no more changes to the input data can be made (annotaions can still be made and components can still be added). If you wish to start over, you must manually delete the data path. Else, reference a new path.

This feature allows you to make these changes to the app on the fly, and you can restart the notebook and it will not change the annotations you have already made.

